<a href="https://colab.research.google.com/github/ju-mk/police_ri/blob/main/police_ri_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset = Traffic stops by police officers at openpolicing.stanford.edu



